# LnZ comparison

In [117]:
import bilby
from funnel.fi_core import get_fi_lnz_list
import numpy as np
import pandas as pd

np.random.seed(42)


class LartillotLikelihood(bilby.Likelihood):
    def __init__(self, dim, v):
        self.dim = dim
        self.v = v
        super().__init__(parameters={f"x{i}": None for i in range(dim)})

    def log_likelihood(self):
        x = np.array(list(self.parameters.values()))
        lnl = -np.power(x, 2) / (2 * self.v)
        if self.dim == 1:
            return lnl[0]
        else:
            return np.sum(lnl)


def get_prior(n_dim):
    pri = bilby.core.prior.PriorDict()
    for i in range(n_dim):
        pri[f'x{i}'] = bilby.core.prior.TruncatedNormal(mu=0, sigma=1, minimum=-1000, maximum=1000)
    return pri


def true_lnz(v, dim):
    return (dim / 2) * (np.log(v) - np.log(1 + v))


def nested_sampling_lnz(v, dim):
    likelihood = LartillotLikelihood(dim, v)
    priors = get_prior(dim)
    result = bilby.run_sampler(
        likelihood=likelihood,
        priors=priors,
        sampler="dynesty",
        nlive=1000,
        label=f"lartillot_dynesty_d{dim}_v{v}",
        clean=False,
        sample='rwalk'
    )
    return (result.log_evidence, result.log_evidence_err)


def parallel_tempering_lnz(v, dim):
    likelihood = LartillotLikelihood(dim, v)
    priors = get_prior(dim)
    result = bilby.run_sampler(
        likelihood=likelihood,
        priors=priors,
        sampler="bilby_mcmc",
        ntemps=10,
        label=f"lartillot_ptmc_d{dim}_v{v}",
        clean=False,
        proposal_cycle='default',
        printdt=10,
        nsamples=2000,
    )
    return (result.log_evidence, result.log_evidence_err)


def __simulate_posterior(v, dim, nsamp=1000):
    likelihood = LartillotLikelihood(dim, v)
    priors = get_prior(dim)
    post = []
    scale = np.sqrt(v / (v + 1))
    for i in range(nsamp):
        params = {f'x{i}': np.random.normal(scale=scale) for i in range(dim)}
        likelihood.parameters.update(params)
        post.append(dict(
            log_likelihood=likelihood.log_likelihood(),
            log_prior=priors.ln_prob(likelihood.parameters),
            **params
        ))
    return pd.DataFrame(post)


def fi_lnz(v, dim, nsamp=1000):
    lnzs, r_vals, samp = get_fi_lnz_list(
        __simulate_posterior(v, dim, nsamp=nsamp),
        r_vals=np.linspace(0.1, 100, 100),
        num_ref_params=1,
        weight_samples_by_lnl=True
    )
    # only keep last 90% of lnzs
    lnzs = lnzs[:, -int(0.9 * nsamp):]
    return np.median(lnzs), np.std(lnzs)


def print_result_dict(results, dim, v):
    print(f"Lartillot {dim}D (v={v:0.2f})")
    print("----------------------------")
    for k, res in results.items():
        if isinstance(res, tuple):
            res = f"{res[0]:0.2f} +/- {res[1]:0.2f}"
        else:
            res = f"{res:0.2f}"
        print(f"{k}: {res}")

## 1D Case

In [ ]:
v, dim = 0.01, 1
results_1d = dict()
results_1d["True"] = true_lnz(v, dim)
results_1d["Nested Sampling"] = nested_sampling_lnz(v, dim)
results_1d["Parallel Tempering"] = parallel_tempering_lnz(v, dim)
results_1d["FI"] = fi_lnz(v, dim)

In [109]:
print_result_dict(results_1d, dim, v)

Lartillot 1D (v=0.01)
----------------------------
True: -2.31
Nested Sampling: -2.29 +/- 0.06
Parallel Tempering: -2.24 +/- 0.02
FI: -2.30 +/- 0.60


## 20D Case

In [ ]:
v, dim = 0.01, 20
results_20d = dict()
results_20d["True"] = true_lnz(v, dim)
results_20d["Nested Sampling"] = nested_sampling_lnz(v, dim)
results_20d["Parallel Tempering"] = parallel_tempering_lnz(v, dim)
results_20d["FI"] = fi_lnz(v, dim)

20:52 bilby INFO    : Running for label 'lartillot_dynesty_d20_v0.01', output will be saved to 'outdir'
/home/avaj040/.cache/pypoetry/virtualenvs/funnel-ZrW66eTN-py3.8/lib/python3.8/site-packages/bilby/core/utils/log.py:73: DeprecationWarning: Accessing attr.__version__ is deprecated and will be removed in a future release. Use importlib.metadata directly to query for attrs's packaging metadata.
  vdict[key] = str(getattr(sys.modules[key], "__version__", "N/A"))
20:52 bilby INFO    : Analysis priors:
20:52 bilby INFO    : x0=TruncatedNormal(mu=0, sigma=1, minimum=-1000, maximum=1000, name=None, latex_label=None, unit=None, boundary=None)
20:52 bilby INFO    : x1=TruncatedNormal(mu=0, sigma=1, minimum=-1000, maximum=1000, name=None, latex_label=None, unit=None, boundary=None)
20:52 bilby INFO    : x2=TruncatedNormal(mu=0, sigma=1, minimum=-1000, maximum=1000, name=None, latex_label=None, unit=None, boundary=None)
20:52 bilby INFO    : x3=TruncatedNormal(mu=0, sigma=1, minimum=-1000, max

1it [00:00, ?it/s]

20:55 bilby INFO    : Written checkpoint file outdir/lartillot_dynesty_d20_v0.01_resume.pickle
20:55 bilby INFO    : Rejection sampling nested samples to obtain 7823 posterior samples
20:55 bilby INFO    : Sampling time: 0:02:59.191276


20:55 bilby INFO    : Summary of results:
nsamples: 7823
ln_noise_evidence:    nan
ln_evidence: -45.256 +/-  0.223
ln_bayes_factor:    nan +/-  0.223

20:55 bilby INFO    : Running for label 'lartillot_ptmc_d20_v0.01', output will be saved to 'outdir'
/home/avaj040/.cache/pypoetry/virtualenvs/funnel-ZrW66eTN-py3.8/lib/python3.8/site-packages/bilby/core/utils/log.py:73: DeprecationWarning: Accessing attr.__version__ is deprecated and will be removed in a future release. Use importlib.metadata directly to query for attrs's packaging metadata.
  vdict[key] = str(getattr(sys.modules[key], "__version__", "N/A"))
20:55 bilby INFO    : Analysis priors:
20:55 bilby INFO    : x0=TruncatedNormal(mu=0, sigma=1, minimum=-1000, maximum=1000, name=None, latex_label=None, unit=None, boundary=None)
20:55 bilby INFO    : x1=TruncatedNormal(mu=0, sigma=1, minimum=-1000, maximum=1000, name=None, latex_label=None, unit=None, boundary=None)
20:55 bilby INFO    : x2=TruncatedNormal(mu=0, sigma=1, minimum=-1

4.80e+01|0:00:10|4.60e+01(AD)|t=inf|n=0|a=0.00|e=0.0e+00%|2.10ms/ev|maxl=-662.57|ETF=-
9.50e+01|0:00:20|9.40e+01(AD)|t=inf|n=0|a=0.00|e=0.0e+00%|2.15ms/ev|maxl=-662.57|ETF=-
1.39e+02|0:00:30|1.39e+02(AD)|t=inf|n=0|a=0.00|e=0.0e+00%|2.28ms/ev|maxl=-662.57|ETF=-
1.85e+02|0:00:40|1.84e+02(AD)|t=inf|n=0|a=0.00|e=0.0e+00%|2.21ms/ev|maxl=-662.57|ETF=-
2.31e+02|0:00:50|2.29e+02(AD)|t=inf|n=0|a=0.00|e=0.0e+00%|2.18ms/ev|maxl=-662.57|ETF=-
2.77e+02|0:01:00|2.77e+02(AD)|t=inf|n=0|a=0.00|e=0.0e+00%|2.19ms/ev|maxl=-662.57|ETF=-
3.24e+02|0:01:10|3.22e+02(AD)|t=inf|n=0|a=0.00|e=0.0e+00%|2.15ms/ev|maxl=-662.57|ETF=-
3.70e+02|0:01:20|3.70e+02(AD)|t=inf|n=0|a=0.00|e=0.0e+00%|2.20ms/ev|maxl=-662.57|ETF=-
4.16e+02|0:01:30|4.15e+02(AD)|t=inf|n=0|a=0.00|e=0.0e+00%|2.18ms/ev|maxl=-662.57|ETF=-
4.64e+02|0:01:40|4.63e+02(AD)|t=inf|n=0|a=0.00|e=0.0e+00%|2.09ms/ev|maxl=-662.57|ETF=-
5.11e+02|0:01:50|5.11e+02(AD)|t=inf|n=0|a=0.00|e=0.0e+00%|2.17ms/ev|maxl=-662.57|ETF=-
5.57e+02|0:02:00|5.56e+02(AD)|t=inf|n=0|a=0

...

21:12 bilby WARNING : Failed to refit chain due to error 2-th leading minor of the array is not positive definite
21:12 bilby WARNING : Failed to refit chain due to error 3-th leading minor of the array is not positive definite
21:12 bilby WARNING : Failed to refit chain due to error 2-th leading minor of the array is not positive definite
21:12 bilby WARNING : Failed to refit chain due to error 2-th leading minor of the array is not positive definite
21:12 bilby WARNING : Failed to refit chain due to error 2-th leading minor of the array is not positive definite
21:12 bilby WARNING : Failed to refit chain due to error 3-th leading minor of the array is not positive definite
21:12 bilby WARNING : Failed to refit chain due to error 2-th leading minor of the array is not positive definite
21:12 bilby WARNING : Failed to refit chain due to error 2-th leading minor of the array is not positive definite
21:12 bilby WARNING : Failed to refit chain due to error 2-th leading minor of the array

4.60e+03|0:17:14|9.97e+02(AD)|t=inf|n=0|a=0.00|e=0.0e+00%|9.73ms/ev|maxl=-662.57|ETF=-


21:12 bilby WARNING : Failed to refit chain due to error 2-th leading minor of the array is not positive definite
21:12 bilby WARNING : Failed to refit chain due to error 3-th leading minor of the array is not positive definite
21:12 bilby WARNING : Failed to refit chain due to error 3-th leading minor of the array is not positive definite
21:12 bilby WARNING : Failed to refit chain due to error 2-th leading minor of the array is not positive definite
21:12 bilby WARNING : Failed to refit chain due to error 2-th leading minor of the array is not positive definite
21:12 bilby WARNING : Failed to refit chain due to error 2-th leading minor of the array is not positive definite
21:12 bilby WARNING : Failed to refit chain due to error 2-th leading minor of the array is not positive definite
21:12 bilby WARNING : Failed to refit chain due to error 4-th leading minor of the array is not positive definite
21:12 bilby WARNING : Failed to refit chain due to error 2-th leading minor of the array

4.61e+03|0:17:25|9.97e+02(AD)|t=inf|n=0|a=0.00|e=0.0e+00%|10.24ms/ev|maxl=-662.57|ETF=-


21:12 bilby WARNING : Failed to refit chain due to error 2-th leading minor of the array is not positive definite
21:12 bilby WARNING : Failed to refit chain due to error 4-th leading minor of the array is not positive definite
21:12 bilby WARNING : Failed to refit chain due to error 2-th leading minor of the array is not positive definite
21:12 bilby WARNING : Failed to refit chain due to error 2-th leading minor of the array is not positive definite
21:12 bilby WARNING : Failed to refit chain due to error 3-th leading minor of the array is not positive definite
21:12 bilby WARNING : Failed to refit chain due to error 2-th leading minor of the array is not positive definite
21:12 bilby WARNING : Failed to refit chain due to error 2-th leading minor of the array is not positive definite
21:12 bilby WARNING : Failed to refit chain due to error 2-th leading minor of the array is not positive definite
21:12 bilby WARNING : Failed to refit chain due to error 3-th leading minor of the array

In [ ]:
print_result_dict(results_20d, dim, v)

## 100D Case

In [ ]:
v, dim = 0.01, 100
results_100d = dict()
results_100d["True"] = true_lnz(v, dim)
results_100d["Nested Sampling"] = nested_sampling_lnz(v, dim)
results_100d["Parallel Tempering"] = parallel_tempering_lnz(v, dim)
results_100d["FI"] = fi_lnz(v, dim)

In [ ]:
print_result_dict(results_100d, dim, v)

## Comparison Plot

In [ ]:
import matplotlib.pyplot as plt

